<a href="https://colab.research.google.com/github/AayushiChoudhary01/qna/blob/main/qna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import random
import json
import tensorflow as tf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install tflearn
!pip install tensorflow
import numpy as np
import tensorflow as tf
!pip uninstall tflearn
!pip install git+https://github.com/MihaMarkic/tflearn.git@fix/is_sequence_missing
import tflearn
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127283 sha256=9615961396b2946f7c99b63cfcaf351285ac2546d1ac0aabd2642ea610a29b9a
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn
Found existing installation: tflearn 0.5.0
Uninstalling tflearn-0.5.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/tflearn-0.5.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tflearn/*
Proceed (Y/n)? 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open("/content/drive/MyDrive/p2data.json") as file:
    data = json.load(file)

In [ ]:
words = []
labels = []
docs_x = []
docs_y = []

In [ ]:
for intent in data["data"]:
    for question in intent["question"]:
        wrds = nltk.word_tokenize(question)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["answer"])

    if intent["answer"] not in labels:
        labels.append(intent["answer"])

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [ ]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

In [ ]:
for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [ ]:
training = np.array(training)
output = np.array(output)

In [ ]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [ ]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        question = labels[results_index]

        for tg in data["data"]:
            if tg['question'] == question :
                answer = tg['answer']

        print(random.choice(answer))

chat()

